In [2]:
import os
import cv2
import pandas as pd
import requests
import face_recognition
from PIL import Image

# File Paths
csv_file = r"C:\Users\cheta\OneDrive\Desktop\sorted_unique_influencers_with_reliability.csv" 
output_folder = "influencer_faces"
os.makedirs(output_folder, exist_ok=True)

data = pd.read_csv(csv_file)

# Function to download and process video
def download_and_process_video(video_url, influencer_id, known_face_encodings):
    try:
        video_response = requests.get(video_url, stream=True)
        video_path = os.path.join(output_folder, f"{influencer_id}.mp4")
        
        with open(video_path, "wb") as video_file:
            for chunk in video_response.iter_content(chunk_size=1024):
                video_file.write(chunk)
        
        cap = cv2.VideoCapture(video_path)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convert to RGB for face recognition
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Detect faces
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            
            for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
                # Check if the face is unique
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
                
                if not any(matches):  # If this is a new unique face
                    known_face_encodings.append(face_encoding)
                    
                    # Crop and save the face
                    face_image = frame[top:bottom, left:right]
                    face_image_path = os.path.join(output_folder, f"{influencer_id}.jpg")
                    cv2.imwrite(face_image_path, face_image)
                    print(f"Saved unique face for Influencer {influencer_id} at {face_image_path}")
                    break
            
            # Stop processing further frames once a unique face is saved
            if influencer_id in [os.path.splitext(f)[0] for f in os.listdir(output_folder)]:
                break
        
        cap.release()
        os.remove(video_path)
    except Exception as e:
        print(f"Error processing video for Influencer {influencer_id}: {e}")

# Known face encodings to avoid duplicates
known_face_encodings = []

# Processing each influencer
for index, row in data.iterrows():
    influencer_id = row["Influencer ID"]
    video_url = row["First Video URL"]
    print(f"Processing video for Influencer {influencer_id}...")
    download_and_process_video(video_url, influencer_id, known_face_encodings)

print("All videos processed and unique faces extracted.")

Processing video for Influencer 15...
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Saved unique face for Influencer 15 at influencer_faces\15.jpg
Processing video for Influencer 6...
Saved unique face for Influencer 6 at influencer_faces\6.jpg
Saved unique face for Influencer 6 at influencer_faces\6.jpg
Processing video for Influencer 97...
Saved unique face for Influencer 97 at influencer_faces\97.jpg
Processing video for Influencer 1...
Saved unique face for Influencer 1 at influencer_faces\1.jpg
Saved unique face for Influencer 1 at influencer_faces\1.jpg
Processing video for Influencer 2...
Saved unique face for Influencer 2 at influencer_faces\2.jpg
Saved unique face for Influencer 2 at influencer_faces\2.jpg
Saved 

In [7]:
import os
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Image as ReportLabImage
from reportlab.lib import colors
from PIL import Image

# File paths
input_csv = r"C:\Users\cheta\OneDrive\Desktop\sorted_unique_influencers_with_reliability.csv"  
faces_folder =  r"C:\Users\cheta\OneDrive\Desktop\influencer_faces"
output_pdf = "influencer_table.pdf"

data = pd.read_csv(input_csv)

table_data = [["Influencer ID", "Image", "Average Performance", "Reliability"]]

# Function to resize image for the PDF
def resize_image(image_path, width, height):
    with Image.open(image_path) as img:
        img.thumbnail((width, height))
        resized_path = f"{image_path}_resized.jpg"
        img.save(resized_path)
        return resized_path

# Iterate over the rows of the CSV
for _, row in data.iterrows():
    influencer_id = row["Influencer ID"]
    avg_performance = row["Average Performance"]
    reliability = row["Reliability"]
    
    # Check if an image exists for the influencer
    face_image_path = os.path.join(faces_folder, f"{influencer_id}.jpg")
    if os.path.exists(face_image_path):
        # Resize the image for better fitting in the table
        resized_image_path = resize_image(face_image_path, 50, 50)
        image = ReportLabImage(resized_image_path, width=50, height=50)
    else:
        image = "No Image"  # Incase of missing images
    table_data.append([influencer_id, image, avg_performance, reliability])

# Creating a PDF document
pdf = SimpleDocTemplate(output_pdf, pagesize=letter)
elements = []

# Creating the table
table = Table(table_data)

# Add style to the table
style = TableStyle([
    ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
    ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
    ("ALIGN", (0, 0), (-1, -1), "CENTER"),
    ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
    ("FONTSIZE", (0, 0), (-1, 0), 12),
    ("BOTTOMPADDING", (0, 0), (-1, 0), 12),
    ("BACKGROUND", (0, 1), (-1, -1), colors.beige),
    ("GRID", (0, 0), (-1, -1), 1, colors.black),
])

table.setStyle(style)

elements.append(table)

pdf.build(elements)

print(f"PDF created successfully: {output_pdf}")

PDF created successfully: influencer_table.pdf
